In [1]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os

# Model and API Key

In [2]:
# Load environment variables.
load_dotenv()

# Set the model name for our LLMs.
OPENAI_MODEL = "gpt-3.5-turbo"
# Store the API key in a variable.
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Basic Query

In [5]:
# Initialize the model.
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model_name=OPENAI_MODEL, temperature=0.3)

# Define a query as a string.
query = "give me two dinners, one featuring chicken and the other salmon"

# Pass the query to the invoke method and print the result.
result = llm.invoke(query)

result

AIMessage(content='1. Chicken stir-fry: \n- Ingredients: chicken breast, mixed vegetables (bell peppers, broccoli, carrots), soy sauce, garlic, ginger, sesame oil, rice\n- Instructions: Cut the chicken into bite-sized pieces and stir-fry in a pan with garlic and ginger. Add in the mixed vegetables and cook until tender. Season with soy sauce and sesame oil. Serve over cooked rice.\n\n2. Baked salmon with lemon dill sauce:\n- Ingredients: salmon fillets, lemon, dill, olive oil, salt, pepper\n- Instructions: Preheat the oven to 400°F. Place the salmon fillets on a baking sheet and drizzle with olive oil. Season with salt, pepper, and chopped dill. Squeeze fresh lemon juice over the top. Bake in the oven for 15-20 minutes or until the salmon is cooked through. Serve with a side of roasted vegetables or a salad.', response_metadata={'token_usage': {'completion_tokens': 188, 'prompt_tokens': 19, 'total_tokens': 207}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_re

In [7]:
print(result.content)

1. Chicken stir-fry: 
- Ingredients: chicken breast, mixed vegetables (bell peppers, broccoli, carrots), soy sauce, garlic, ginger, sesame oil, rice
- Instructions: Cut the chicken into bite-sized pieces and stir-fry in a pan with garlic and ginger. Add in the mixed vegetables and cook until tender. Season with soy sauce and sesame oil. Serve over cooked rice.

2. Baked salmon with lemon dill sauce:
- Ingredients: salmon fillets, lemon, dill, olive oil, salt, pepper
- Instructions: Preheat the oven to 400°F. Place the salmon fillets on a baking sheet and drizzle with olive oil. Season with salt, pepper, and chopped dill. Squeeze fresh lemon juice over the top. Bake in the oven for 15-20 minutes or until the salmon is cooked through. Serve with a side of roasted vegetables or a salad.


In [8]:
# Define a function to use two ingredients as inputs for the query sent to our LLM.
def dinner_recipe(food1, food2):
    # Initialize the model.
    llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model_name=OPENAI_MODEL, temperature=0.3)
    # Construct the query using the parameters.
    query = f"give me two dinners, one featuring {food1} and the other {food2}"
    # Pass the query to the invoke method which will make a prediction based on the query, and print the result.
    result = llm.invoke(query)
    print(result.content)
    


# Ask the user for a main ingredient for each day.
food1 = input("tell me your ingredient 1: ")
food2 = input("tell me your ingredient 2: ")


# Call the function with the two ingredients.
dinner_recipe(food1, food2)

tell me your ingredient 1: chicken
tell me your ingredient 2: salmon
1. Chicken stir-fry: 
- Ingredients: chicken breast, mixed vegetables (bell peppers, broccoli, carrots), soy sauce, garlic, ginger, sesame oil, rice
- Instructions: Cut chicken into bite-sized pieces and stir-fry in a pan with garlic and ginger. Add in mixed vegetables and cook until tender. Pour in soy sauce and sesame oil, then serve over cooked rice.

2. Baked salmon with lemon dill sauce: 
- Ingredients: salmon fillets, lemon, fresh dill, olive oil, salt, pepper
- Instructions: Preheat oven to 400°F. Place salmon fillets on a baking sheet lined with foil. Drizzle with olive oil and season with salt and pepper. Slice lemon and place on top of salmon. Bake for 15-20 minutes or until salmon is cooked through. In a small bowl, mix together chopped dill, lemon juice, and a little olive oil to make a sauce. Serve salmon with the lemon dill sauce on top.


# Chains

In [9]:
# Additional imports for specific chain we'll use.
from langchain.chains import LLMMathChain

In [13]:
# Initialize the model.
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model_name=OPENAI_MODEL, temperature=0.0)

# Create a math chain for performing calculations.
chain = LLMMathChain.from_llm(llm=llm, verbose=True)

# Set the input query as a text description of a math problem.
query = {"question" : "What is the sum of 20 and the cubed root 24?"}

# Run the chain using the query as input and print the result.
result = chain.invoke(query)
print(result['answer'])



> Entering new LLMMathChain chain...
What is the sum of 20 and the cubed root 24?```text
20 + 24**(1/3)
```
...numexpr.evaluate("20 + 24**(1/3)")...

Answer: 22.884499140614817
> Finished chain.
Answer: 22.884499140614817


In [14]:
result

{'question': 'What is the sum of 20 and the cubed root 24?',
 'answer': 'Answer: 22.884499140614817'}

# Combining Chains

In [15]:
# Additional imports for specific chains we'll use.
from langchain.chains import LLMChain
from langchain.chains import LLMMathChain
from langchain.chains import SimpleSequentialChain
from langchain.prompts import ChatPromptTemplate

In [22]:
# Initialize the LLM for text.
chat_llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model_name=OPENAI_MODEL, temperature=1.2)

# Initialize the LLM for math.
math_llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model_name=OPENAI_MODEL, temperature=0.0)

# Create a chat chain for creating text.
simple_prompt = ChatPromptTemplate.from_template("{query}")
chat_chain = LLMChain(llm=chat_llm, prompt=simple_prompt)

# Create a math chain for performing calculations.
math_chain = LLMMathChain.from_llm(llm=math_llm)

# Construct the simple sequential chain from the two other chains.
chain = SimpleSequentialChain(
    chains = [chat_chain, math_chain],
    verbose=True
)

# Set the input query for the first chain in the sequence.
query = {"input" : "Please write a simple math word problem requiring multiplication"}

# Run the sequential chain using the query as the first input. Print the result.
result = chain.invoke(query)
print(result['output'])



> Entering new SimpleSequentialChain chain...
Tom and his friends went to the carnival. Each ride costs $3 per person and they went on 5 different rides. How much money did Tom and his friends spend in total on rides at the carnival?
Answer: 15

> Finished chain.
Answer: 15


In [23]:
result

{'input': 'Please write a simple math word problem requiring multiplication',
 'output': 'Answer: 15'}

In [28]:
# Additional imports for specific chains we'll use.
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate
from langchain.chains.constitutional_ai.base import ConstitutionalChain
from langchain.chains.constitutional_ai.models import ConstitutionalPrinciple

In [45]:
# Initialize the model.
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model_name=OPENAI_MODEL, temperature=0.3)

# Create a chat chain for creating text.
simple_prompt = ChatPromptTemplate.from_template("{query}")
chat_chain = LLMChain(llm=llm, prompt=simple_prompt)

In [46]:
# Create a principle for our constitutional chain.
principle = ConstitutionalPrinciple(
    name = "Fear of Dogs",
    critique_request="The model should not include in stories it writes",
    revision_request="Modify the story to be about animals other than dogs"
)
principle

# principle = ConstitutionalPrinciple(
#     name = "Only Cat",
#     critique_request="The model should not include in stories it writes",
#     revision_request="Modify the story to be about animals other than dogs"
# )
# principle

ConstitutionalPrinciple(critique_request='The model should not include in stories it writes', revision_request='Modify the story to be about animals other than dogs', name='Fear of Dogs')

In [47]:
# Create a constitutional chain for ensuring the story does not include dogs.
constitutional_chain = ConstitutionalChain.from_llm(
    chain=chat_chain,
    constitutional_principles=[principle],
    llm=llm,
    verbose=True
)

In [55]:
# constitutional_chain

In [49]:
# Set the input query for the chat chain.
query = {"query" : "Please give me the main events of a story about three household pets."}

# Run the constitutional chain using the query as the first input.
result = constitutional_chain.invoke(query)
print(result['output'])



> Entering new ConstitutionalChain chain...
Initial response: 1. Three household pets - a cat named Whiskers, a dog named Max, and a bird named Tweetie - live together in a cozy home with their owner, a kind elderly woman named Mrs. Jenkins.

2. One day, Mrs. Jenkins falls ill and is taken to the hospital, leaving the pets alone in the house for the first time. They are confused and worried about her absence.

3. As the days pass, the pets learn to work together to take care of themselves and the house. Whiskers catches mice that try to invade the kitchen, Max barks at strangers to protect the house, and Tweetie chirps happily to keep everyone's spirits up.

4. Despite their efforts, the pets miss Mrs. Jenkins terribly and long for her return. They spend their days waiting by the window, hoping to see her car pull into the driveway.

5. Finally, after a week of being away, Mrs. Jenkins returns home from the hospital. The pets are overjoyed to see her and shower her with love and affe

In [ ]:
# prompt = f"""
# You're an AI assistant who likes to help people on internal documentation. 

# Please answer the question given the following information:

# CONTEXT: {context}

# QUERY: {query}
# """